In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import citipy
from geoviews import dim
import json
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_weather_data.csv")

# Display sample data
city_data_df.head()

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,albany,42.6001,-73.9662,20.30,77,48,0.38,US,1722592873
1,2,vegreville,53.5001,-112.0518,15.17,96,84,3.60,CA,1722592905
2,3,thompson,55.7435,-97.8558,14.09,82,0,3.09,CA,1722592723
3,4,grytviken,-54.2811,-36.5092,-1.23,98,100,4.80,GS,1722592697
4,5,castro,-24.7911,-50.0119,10.29,98,16,2.52,BR,1722592909


# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [19]:
# Configure the map
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = dim("Humidity")*0.1,
    alpha=0.5,
    scale = 2,
    color = "City",
    hover_cols = ["City", "Humidity"]
)
# Display the map plot
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [35]:
# Save the map to an HTML file using holoviews
hv.save(city_map, "city_map.html", backend='bokeh')

# Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [20]:
filtered_cities = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# drop NA
filtered_cities_clean = filtered_cities.dropna()

# print ressult
print(filtered_cities_clean.to_string(index=False))

 ID              City      Lat       Lng  Max Temp  Humidity  Cloudiness  Wind Speed Country       Date
 15            bethel  41.3712  -73.4140     22.93        88           0        0.00      US 1722592770
 21           stanley  54.8680   -1.6985     22.38        66           0        3.60      GB 1722592874
 26           santana  -0.0583  -51.1817     24.10        94           0        1.54      BR 1722592935
 43   remire-montjoly   4.9167  -52.2667     24.02       100           0        1.03      GF 1722592955
123           lincoln  40.8000  -96.6670     21.12        92           0        2.06      US 1722592600
184         santa ana  33.7456 -117.8678     23.01        83           0        0.00      US 1722592825
208        ga-kgapane -23.6333   30.2167     24.84        32           0        2.44      ZA 1722593167
293            laguna  38.4210 -121.4238     23.63        60           0        2.06      US 1722593272
303      saint-pierre -21.3393   55.4781     25.87        53    

# Step 3: Create a new DataFrame called hotel_df

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print(hotel_df)

                  City Country      Lat       Lng  Humidity Hotel Name
14              bethel      US  41.3712  -73.4140        88           
20             stanley      GB  54.8680   -1.6985        66           
25             santana      BR  -0.0583  -51.1817        94           
42     remire-montjoly      GF   4.9167  -52.2667       100           
122            lincoln      US  40.8000  -96.6670        92           
183          santa ana      US  33.7456 -117.8678        83           
207         ga-kgapane      ZA -23.6333   30.2167        32           
292             laguna      US  38.4210 -121.4238        60           
302       saint-pierre      RE -21.3393   55.4781        53           
307             balaka      MW -14.9793   34.9558        24           
399          pinehurst      US  35.1954  -79.4695        91           
400              zomba      MW -15.3860   35.3188        27           
403       south venice      US  27.0531  -82.4243        95           
406   

# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Get only the first result
    "apiKey": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
      # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" 
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
stanley - nearest hotel: Hotel 52
santana - nearest hotel: Hotel Teodoro
remire-montjoly - nearest hotel: Complexe Belova
lincoln - nearest hotel: New Victorian Suites Lincoln
santa ana - nearest hotel: Holiday Inn Express
ga-kgapane - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
saint-pierre - nearest hotel: Tropic Hotel
balaka - nearest hotel: Zembani Lodge
pinehurst - nearest hotel: The Magnolia Inn
zomba - nearest hotel: Zomba City Council Lodge
south venice - nearest hotel: Kon-Tiki Motel
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
coatesville - nearest hotel: Courtyard
nova olimpia - nearest hotel: Olímpia Hotel
medina de rioseco - nearest hotel: Hotel Vittoria Colonna
grantsville - nearest hotel: No hotel found
colby - nearest hotel: Budget Inn & Suites
caruthersville - nearest hotel: Heritage Place & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
14,bethel,US,41.3712,-73.4140,88,Hampton Inn Danbury
20,stanley,GB,54.8680,-1.6985,66,Hotel 52
25,santana,BR,-0.0583,-51.1817,94,Hotel Teodoro
42,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
122,lincoln,US,40.8000,-96.6670,92,New Victorian Suites Lincoln
183,santa ana,US,33.7456,-117.8678,83,Holiday Inn Express
207,ga-kgapane,ZA,-23.6333,30.2167,32,No hotel found
292,laguna,US,38.4210,-121.4238,60,Holiday Inn Express & Suites
302,saint-pierre,RE,-21.3393,55.4781,53,Tropic Hotel
307,balaka,MW,-14.9793,34.9558,24,Zembani Lodge


# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map

In [23]:
# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = dim("Humidity")*0.2,
    alpha=0.5,
    scale = 2,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [27]:
# Save the map to an HTML file using holoviews
hv.save(hotel_map, "hotel_map.html", backend='bokeh')